In [1]:
import numpy as np
import pandas as pd
import math
from xml.dom import minidom
from xml.etree import cElementTree as ElementTree
import os
import nltk
import pickle
import csv
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
import timeit
# Ranked Retrieval and Document Vectorization

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\diego\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\diego\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\diego\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
def documentReader(path, queries = False):
    """
    DocString
    :return: Nothing
    """
    documents_path = os.path.join(os.getcwd(), path)
    documentos = {}
    for filename in os.listdir(documents_path):
        file_path = os.path.join(documents_path, filename)
        xmldoc = minidom.parse(file_path)
        id = xmldoc.getElementsByTagName('public')[0].attributes['publicId'].value
        title = '' if queries else xmldoc.getElementsByTagName('fileDesc')[0].attributes['title'].value
        data = next(ElementTree.parse(file_path).iter('raw')).text
        documentos[id] = (title + ' ' + data).replace(u'\xa0', u' ').replace('\n', ' ')

    return documentos
documentos = documentReader('docs/docs-raw-texts')
NRO_DOCS = len(documentos)
DOCS_IDs = list(documentos.keys())
print(list(documentos.items())[0])

('d001', 'William Beaumont and the Human Digestion William Beaumont and the Human Digestion.  William Beaumont: Physiology of digestion Image Source.  On November 21, 1785, US-American surgeon William Beaumont was born. He became best known as “Father of Gastric Physiology” following his research on human digestion. William Beaumont was born in Lebanon, Connecticut and became a physician. He served as a surgeon’s mate in the Army during the War of 1812. He opened a private practice in Plattsburgh, New York, but rejoined the Army as a surgeon in 1819. Beaumont was stationed at Fort Mackinac on Mackinac Island in Michigan in the early 1820s when it existed to protect the interests of the American Fur Company. The fort became the refuge for a wounded 19-year-old French-Canadian fur trader named Alexis St. Martin when a shotgun went off by accident in the American Fur Company store at close range June 6th, 1822. St. Martin’s wound was quite serious because his stomach was perforated and se

In [3]:
def tokenization(documentos):
    """
    :param documentos:
    :return:
    """

    nltk_stop_words_en = set(nltk.corpus.stopwords.words("english"))
    wordnet_lemmatizer = nltk.stem.WordNetLemmatizer()

    word_tok = {key: nltk.word_tokenize(doc) for key, doc in documentos.items()}
    word_tok_sw = {key: [token for token in doc
                         if token.isalnum() and token not in nltk_stop_words_en]
                   for key, doc in word_tok.items()}
    nltk_lemmaList = {key: [wordnet_lemmatizer.lemmatize(word) for word in doc] for key, doc in word_tok_sw.items()}

    return nltk_lemmaList
tokenized_docs = tokenization(documentos)
print(list(tokenized_docs.items())[0])

('d001', ['William', 'Beaumont', 'Human', 'Digestion', 'William', 'Beaumont', 'Human', 'Digestion', 'William', 'Beaumont', 'Physiology', 'digestion', 'Image', 'Source', 'On', 'November', '21', '1785', 'surgeon', 'William', 'Beaumont', 'born', 'He', 'became', 'best', 'known', 'Father', 'Gastric', 'Physiology', 'following', 'research', 'human', 'digestion', 'William', 'Beaumont', 'born', 'Lebanon', 'Connecticut', 'became', 'physician', 'He', 'served', 'surgeon', 'mate', 'Army', 'War', '1812', 'He', 'opened', 'private', 'practice', 'Plattsburgh', 'New', 'York', 'rejoined', 'Army', 'surgeon', '1819', 'Beaumont', 'stationed', 'Fort', 'Mackinac', 'Mackinac', 'Island', 'Michigan', 'early', '1820s', 'existed', 'protect', 'interest', 'American', 'Fur', 'Company', 'The', 'fort', 'became', 'refuge', 'wounded', 'fur', 'trader', 'named', 'Alexis', 'Martin', 'shotgun', 'went', 'accident', 'American', 'Fur', 'Company', 'store', 'close', 'range', 'June', '6th', '1822', 'Martin', 'wound', 'quite', 'ser

In [4]:
def indexReader():
    """
    Reads the inverted index created in the distributed_inverted_index.ipynb notebook
    The path from where it reads the file is docs/inverted_index.pkl
    :return: the inverted index, represented in a dictionary where the keys are the terms
    and the values is also a dictionary that contains the frecuency of documents
    that contain the term, and the posting. The posting is an array itself
    that contains the document id, and the term's frequency in that document.
    {'term': {'freq': df, 'posting':[[doc1, tf1],[doc2, tf2],...,[docn, tfn]}}
    """
    with open('docs/inverted_index.pkl', 'rb') as index:
        return pickle.load(index)

invertedIndex = indexReader()
print(invertedIndex['William'])


{'freq': 46, 'posting': [[1, 4], [15, 5], [28, 3], [35, 2], [55, 2], [56, 4], [69, 4], [88, 3], [91, 1], [92, 1], [95, 1], [98, 2], [102, 3], [106, 1], [109, 1], [111, 1], [129, 1], [136, 6], [138, 3], [147, 1], [175, 1], [179, 2], [180, 1], [189, 2], [190, 1], [191, 1], [197, 1], [212, 1], [230, 1], [241, 2], [254, 1], [257, 1], [266, 2], [272, 1], [273, 6], [274, 1], [289, 1], [291, 1], [294, 1], [299, 1], [300, 1], [309, 1], [310, 3], [320, 4], [323, 1], [330, 6]]}


In [5]:
def tfidfWeightedVector(invertedIndex):
    """
    Recibe el inverted index y lo usa para crear el weighted vector
    :param invertedIndex: Inverted index calculada antes
    :return: Matriz |V|x|D|. Donde |V| es el tamaño de mi vocabulario y |D| es el número de documentos.
    Cada fila es la representación en tfidf de un término
    """

    weightedVectorMatrix = []
    index = []
    for term, term_dict in invertedIndex.items():
        weighted_vector = np.zeros(NRO_DOCS)
        freq = term_dict['freq']
        index.append(term)
        for id, t_freq in term_dict['posting']:
            tfidf = np.log(1 + t_freq) * np.log10(NRO_DOCS / freq)
            weighted_vector[ id - 1] = tfidf

        weightedVectorMatrix.append(weighted_vector)


    weighted_vector_df = pd.DataFrame.from_records(data=weightedVectorMatrix, index=index, columns=DOCS_IDs)
    return weighted_vector_df, index

weighted_vector_df, term_index = tfidfWeightedVector(invertedIndex)
weighted_vector_df.tail()

,d001,d002,d003,d004,d005,d006,d007,d008,d009,d010,...,d322,d323,d324,d325,d326,d327,d328,d329,d330,d331
Čech,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Łukasiewicz,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Šufflay,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
λ,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
π,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [6]:
print(f'Matriz tfidf de dimension {weighted_vector_df.shape}')

Matriz tfidf de dimension (18840, 331)


In [7]:
def norma(v):
    suma = sum(v[i]**2 for i in range(len(v)))
    return math.sqrt(suma)

def dot_product(v1, v2):
    product = sum( v1[0][i]*v2[i][0] for i in range(len(v2)) )
    return product

def cosine_Similarity(doc_vec1, doc_vec2):
    # print('.')
    return (dot_product(doc_vec1, doc_vec2)) / (norma(doc_vec1.flatten()) * norma(doc_vec2.flatten()))

def cosine_Similarity_normQ(query, doc):
    return dot_product(query, doc) / norma(doc.flatten())

# HAcer ejemplo a mano a ver si sirve

In [8]:
#Desde acá
start_time = timeit.default_timer()
queries = documentReader('docs/queries-raw-texts', True)
print(list(queries.items())[0])

('q01', ' Fabrication of music instruments')


In [9]:
tokenized_queries = tokenization(queries)
print(list(tokenized_queries.items())[0])

('q01', ['Fabrication', 'music', 'instrument'])


In [10]:

def vectorize_queries(queries, term_index):
    """
    Vectoriza las queries tal forma que cada una sea un vector unitario con la misma dimensión del tamaño
    de vocabulario encontrado en el corpus de los documentos.
    :param queries: Diccionario con pares key, value. Donde key es el id de cada query y el value es la
    tokenización de dicha query.
    :param term_index: Lista con los ids de las queries. i.e. ['q01', 'q02', ...]
    :return:
    """

    vector_queries = []
    queries_index = []
    for id, query in queries.items():
        queries_index.append(id)
        query_vector = np.zeros(len(term_index)) #Vector de ceros de dimensión V
        len_query = len(query)
        for term in query:
            try:
                index = term_index.index(term)
                query_vector[index] = 1 / math.sqrt(len_query) #Pone en 1 la dimensión del vector correpondiente al termino en term
            except:
                print(f'El término "{term}" de la query {id} no está en los docs')

        vector_queries.append(query_vector)
    return vector_queries, queries_index

vector_queries, queries_index = vectorize_queries(tokenized_queries, term_index)

# vector_queries[0][:1000]


El término "Fabrication" de la query q01 no está en los docs
El término "Computers" de la query q24 no está en los docs
El término "WWII" de la query q25 no está en los docs
El término "Religious" de la query q38 no está en los docs
El término "Personalities" de la query q41 no está en los docs
El término "Campaign" de la query q44 no está en los docs
El término "Friends" de la query q45 no está en los docs


In [11]:
matrix_queries = pd.DataFrame.from_records(data=vector_queries, index=queries_index, columns=term_index)
matrix_queries.head()

,0,01,1,10,100,1000,10000,101,102,1024,...,élèves,élémentaire,émigré,émigrés,über,Čech,Łukasiewicz,Šufflay,λ,π
q01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
q02,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
q03,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
q04,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
q06,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [12]:
print(matrix_queries.iloc[1].sum())

1.7320508075688776


In [13]:
def getCosineSimilarity(queries, documents, query_index, docs_index):
    """
    Recibe la matriz tfidf de documentos y la de queries y saca las matriz de similitud usando coseno como
    métrica de similitud
    :param queries: Matriz |Q|x|V| donde |V| es el tamaño del vocabulario y |Q| el número de queries.
    Cada vector-fila es unitario y tiene 0 en todoas las columnas con términos que no hacen parte de la query
    que representa la fila en cuestión.
    :param documents: Matriz |V|x|D|, Donde |D| es el número de documentos. Cada fila es la representación tfidf de un
    término.
    :param query_index: Lista con los ids de las queries. i.e. ['q01', 'q02', ...]
    :param docs_index: Lista con los ids de los docs. i.e.  ['d01', 'd02', ...]
    :return:Retorna la matriz de similitud entre las queries y los documentos
    """

    similarity_matrix = []

    for query in query_index:
        print(query, end=' - ')
        row_query = queries.loc[[query]].values
        query_doc_sim = []
        for document in docs_index:
            col_document = documents[[document]].values
            cos_sim = cosine_Similarity_normQ(row_query, col_document)
            query_doc_sim.append(cos_sim)

        similarity_matrix.append(query_doc_sim)

    return similarity_matrix

similarity_matrix = getCosineSimilarity(matrix_queries, weighted_vector_df, queries_index, DOCS_IDs)

print(len(similarity_matrix))


q01 - q02 - q03 - q04 - q06 - q07 - q08 - q09 - q10 - q12 - q13 - q14 - q16 - q17 - q18 - q19 - q22 - q23 - q24 - q25 - q26 - q27 - q28 - q29 - q32 - q34 - q36 - q37 - q38 - q40 - q41 - q42 - q44 - q45 - q46 - 35


 ## Save cosine similarity Matrix


In [14]:
with open('docs/cos_sim_matrix', 'wb') as picklefile:
    pickle.dump(similarity_matrix,picklefile)

## Read cosine similarity Matrix

In [15]:
with open('docs/cos_sim_matrix', 'rb') as matrix:
    similarity_matrix = pd.DataFrame.from_records(pickle.load(matrix), index=queries_index, columns=DOCS_IDs)


In [16]:
similarity_matrix.head(10)

,d001,d002,d003,d004,d005,d006,d007,d008,d009,d010,...,d322,d323,d324,d325,d326,d327,d328,d329,d330,d331
q01,0.000000,0.000000,0.000000,0.022011,0.000000,0.038817,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.024966,0.000000,0.000000
q02,0.008216,0.038387,0.005380,0.006699,0.010479,0.000000,0.017204,0.000000,0.000000,0.008788,...,0.000000,0.008176,0.007667,0.000000,0.0,0.012593,0.005852,0.000000,0.006138,0.000000
q03,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000
q04,0.019206,0.000000,0.004159,0.000000,0.022856,0.013380,0.006650,0.006181,0.011784,0.057539,...,0.000000,0.000000,0.004156,0.000000,0.0,0.011717,0.000000,0.000000,0.029640,0.005930
q06,0.000000,0.000000,0.018593,0.033279,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.210759,0.000000,0.000000
q07,0.000000,0.000000,0.000000,0.134515,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000
q08,0.000000,0.000000,0.000000,0.000000,0.072756,0.011019,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.008476,0.000000,0.000000,0.0,0.000000,0.000000,0.012359,0.009073,0.000000
q09,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000
q10,0.011974,0.000000,0.004947,0.000000,0.000000,0.000000,0.000000,0.000000,0.008843,0.011303,...,0.011416,0.012240,0.004943,0.009777,0.0,0.014573,0.008529,0.012183,0.016901,0.007052
q12,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.022922,...,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000


### Retrieve ordered docs per query

In [17]:
def retrieve_docs(similarity_matrix, query_index):
    """
    Para cuada query se aplica el método de cosine similarity y se devuelve una lista ordenada con ids de docs
    relevantes para esa query ordenados por el número del id.
    :param similarity_matrix: Cosine similarity matrix calculada previamente
    :param query_index: Lista con los ids de las queries. i.e. ['q01', 'q02', ...]
    :return: El diccionario con pares key, value. Key = id del query. Value lista con resultados relevantes de docs
    :según el cosine similarity.
    """
    results = {}
    results_with_scores = {}
    for query in query_index:
        order = similarity_matrix.loc[[query]].sort_values(by=query, axis=1, ascending=False, inplace=False)
        relevant = order.loc[:, (order != 0 ).any(axis=0)]
        results[query] = relevant.columns.values.tolist()
        results_with_scores[query] = relevant.to_dict(orient='list')
    return results, results_with_scores

results, results_with_scores = retrieve_docs(similarity_matrix, queries_index)
print(results['q01'][:5])

# Hasta acá

['d254', 'd016', 'd153', 'd209', 'd186']


In [18]:
stop_time = timeit.default_timer()


print(stop_time - start_time)

304.68175360000004


In [19]:
def writeScoreFile(RRDV):
    """
    Writes the RRI-queries_results.tsv that contains the score of each query
    :param RRDV: Dictionary with the scores, {'qYY': {'dXXX': score1, 'dXXX': score2, ..., 'dXXX': score3 } }
    """
    file_path = os.path.join(os.getcwd(), 'docs/answer_files/RRDV-queries_results.tsv')
    with open(file_path, 'wt') as out_file:
        tsv_writer = csv.writer(out_file, delimiter='\t')
        for query_id, scores in RRDV.items():
            scores_list = ""
            for doc, score in scores.items():
                scores_list+= doc +":"+str(round(score[0],4))+","
            tsv_writer.writerow([query_id,scores_list[:-1]])


writeScoreFile(results_with_scores)
print("Archivo escrito")

Archivo escrito


## Evaluation

In [20]:
def read_judgemnts_file():
    """
    Lee el archivo de relevancia de los jueces
    :return: Diccionario con pares key: value, donde el key es el id de cada query y el value
    es otro doccionario con las ids de los docs relevantes para esa query ordenados en forma creciente.
    """
    document_path = os.path.join(os.getcwd(), 'docs/relevance-judgments.tsv')
    tsv_file = open(document_path)
    read_tsv = csv.reader(tsv_file, delimiter="\t")
    relevance = {}
    for row in read_tsv:
        documents = row[1].split(',')
        query_relevance = {pair.split(':')[0] : pair.split(':')[1] for pair in documents }
        query_relevance = dict(sorted(query_relevance.items(), key=lambda item: item[0]))
        relevance[row[0]] = query_relevance
    return relevance


relevance = read_judgemnts_file()
print(relevance['q01'])

{'d016': '5', 'd186': '4', 'd254': '5'}


In [21]:
def make_binary_result(results, relevant_res):
    """
    Este método toma los resultados crudos obtenidos para las queries (Para cada query la lista de documentos ordenaos
    por relevancia), devuelve 3 representaciones de estos resultados. La primera es la representacion binaria at K.
    Que es del mismo tamaño que el número de documentos relevantes. La segunda es esta misma lista pero con la escala
    dada por el archivo de evaluación. La tercera está destinada al cálculo del MAP, tiene la representación binaria
    hasta que salgan todos los documentos relevantes o simplemente de todos los documentos, además en su segundo
    componente tiene el número de documentos relevantes que deberían salir en los resultados según el archivo de
    evaluación.
    :param results: Diccionario con resultados crudos de cada query. Ej: {'q01': ['d254', 'd016', 'd153', ...]}
    :param relevant_res: Las 3 representaciones antes mencionadas
    :return:
    """
    bin_relevant = {}
    rel_scale_repr = {}
    map_relevant_docs = {}
    for query, relevant_docs in relevant_res.items():
        bin_repr = []
        scaled_repr = []
        map_repr = []
        M = len(relevant_docs)
        for doc_id, rel_scale in relevant_docs.items():
            bin = 1 if doc_id in results[query][:M] else 0
            bin_repr.append(bin)
            scaled_repr.append(bin * int(rel_scale))
        i = 0
        for doc_id in results[query]:
            if i < M:
                map_bin = 1 if doc_id in relevant_res[query] else 0
                i += map_bin
                map_repr.append(map_bin)
        bin_relevant[query] = bin_repr
        rel_scale_repr[query] = scaled_repr
        map_relevant_docs[query] = [map_repr, M]
    return bin_relevant, rel_scale_repr, map_relevant_docs

bin_results, scaled_results, map_relevant_docs = make_binary_result(results, relevance)
print(bin_results['q01'])
print(scaled_results['q01'])

[1, 0, 1]
[5, 0, 5]


In [22]:
print('Primeros 5 documentos devueltos como relevantes para q01: \n', results['q01'][:5])
print('Documentos relevantes para q01 según jueces: \n' , relevance['q01'])
print('Representación binaria de q01, hasta el último doc relevante: \n' ,map_relevant_docs['q01'])

Primeros 5 documentos devueltos como relevantes para q01: 
 ['d254', 'd016', 'd153', 'd209', 'd186']
Documentos relevantes para q01 según jueces: 
 {'d016': '5', 'd186': '4', 'd254': '5'}
Representación binaria de q01, hasta el último doc relevante: 
 [[1, 1, 0, 0, 1], 3]


### Definition of IR metrics functions

In [23]:
def precision_at_k(relevance: list, k: int):
    """
    DocString
    :return: Nothing
    """
    if k == 0:
        return 0
    l = np.array(relevance[:k]).sum()/k
    return l

def recall_at_k(relevance: list, nr_relevant: int, k: int):
    """
    DocString
    :return: Nothing
    """
    l = np.array(relevance[:k]).sum()/nr_relevant
    return l

def average_precision(relevance):
    """
    DocString
    :return: Nothing
    """

    length = len(relevance[0])
    sum = 0
    for i in range(length):
        if relevance[0][i]:
            sum += precision_at_k(relevance[0], i+1)
    if np.array(relevance[0]).sum()==0:
        return 0
    else:
        return sum / relevance[1]

def mean_avg_precision(l):
    """
    DocString
    :return: Nothing
    """
    mean = np.array([ average_precision(lista) for lista in l]).mean()
    return mean

mean_avg_precision([[[0, 1, 0, 1, 1, 1, 1], 5]])

0.5961904761904762

In [24]:
def dcg_at_k(relevance, k: int):
    """
    Calcula el DCG at K de un vector binario representando los resultados relevantes para una query.
    :param relevance: Vector binario
    :return: DCG at K de nuestra query
    """

    sum = 0
    i =  0
    for rel_i in relevance[: k]:
        i+= 1
        sum += rel_i/np.log2(max(i, 2))

    return sum

dcg_at_k([4, 4, 3, 0, 0, 1, 3, 3, 3, 0], 6)

def ndcg_at_k(relevance, k):
    """
    Calcula el ndcg at k de un vector binario
    :return: NDCG at K.
    """
    rel_sorted = sorted(relevance, reverse=True)
    max = dcg_at_k(rel_sorted, k)
    real = dcg_at_k(relevance, k)

    return real/ max


ndcg_at_k([4, 4, 3, 0, 0, 1, 3, 3, 3, 0], 6)

0.7424602308163405

In [25]:
print(recall_at_k(bin_results['q01'], 3, 3))


0.6666666666666666


## Compute Evaluation Metrics for each query

In [26]:
def evaluation_metric(bin_queries, query_index, scaled_results):
    """

    :param bin_queries: Diccionario con valores {query Key: vector}, donde el vector corresponde a una lista
    con la representación binaria de un de los resultados encontrados para una query con relación a los dados
    en el archivo de evaluación. Ej, para q01, los relevantes son: d186,d254,d016. El RRDV devuelve d254, d016,
    d153. Por ende, la representación binaria de q01, en el orden del archivo de evaluación es: [0, 1, 1]
    :param query_index: Lista con los ids de las queries. ['qo1', 'qo2', ...]
    :param scaled_results: Representación escalada de los resultados de las queries usando la escala dada en el
    archivo de evaluación. Ej, q01 pasa de [0, 1, 1] a [0, 5, 5]
    :return: Un dataframe con el cálculo del P@M, r@M y NDCG@M para cada query
    """
    COLUMNS = ['P@M', 'R@M', 'NDCG@M']
    records = []
    for query, bin_vec in bin_queries.items():
        scaled = scaled_results[query]
        M = len(bin_vec)
        pm = precision_at_k(bin_vec, M)
        rm = recall_at_k(bin_vec, M, M)
        ndcg = ndcg_at_k(scaled, M)
        records.append([pm, rm, ndcg])
        
    return pd.DataFrame.from_records(records, index=query_index, columns=COLUMNS)
        
metrics = evaluation_metric(bin_results, queries_index, scaled_results)
metrics

<ipython-input-24-bfbe2ec5f786>:27: RuntimeWarning: invalid value encountered in double_scalars
  return real/ max


,P@M,R@M,NDCG@M
q01,0.666667,0.666667,0.815465
q02,0.454545,0.454545,0.536073
q03,1.000000,1.000000,0.874220
q04,0.857143,0.857143,0.933486
q06,1.000000,1.000000,0.863930
q07,0.250000,0.250000,1.000000
q08,0.750000,0.750000,0.842140
q09,0.833333,0.833333,0.880115
q10,0.500000,0.500000,0.642423
q12,0.750000,0.750000,0.797833


### MAP

In [27]:
def overall_map(map_relevant_docs):
    """
    Función que calcula el MAP de los resultados de las queries.
    :param map_relevant_docs: Vector binario de las queries asegurandose de que aparezcan todos los documentos relevantes
    :return: El Mean average precision de los resultados de las queries.
    """
    matrix = [vector for key, vector in map_relevant_docs.items() ]
    return mean_avg_precision(matrix)

print(f'MAP resultante de todas las queries: {overall_map(map_relevant_docs)}')

MAP resultante de todas las queries: 0.6503449127718149
